# Introdução a Modelos Baseados em Transfomers
#### Aula Prática

### Descriçao

---
1. O objetivo deste notebook é aprender a usar o transformers e suas pipelines na prática, via código.

2. Utilizaremos o conjunto de dados proposto no [hugging face](https://huggingface.co/) e também a api do [OPENAI](https://platform.openai.com)
---

# Instalação dos pacotes

# Instalando as bibliotecas


In [ ]:
!pip install openai==0.28

In [ ]:
!pip install transformers  beautifulsoup4 requests


# Declarando as bibliotecas

In [ ]:
from transformers import pipeline, BertTokenizerFast, EncoderDecoderModel
import torch
import openai
import requests
from bs4 import BeautifulSoup

# Raspando dados da internet

In [ ]:
def extraindo_texto_da_web(url):
    response = requests.get(url)
    response.raise_for_status()  # Verifica se o request foi bem-sucedido
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extrai todo o texto visível da página
    paragraphs = soup.find_all('p')
    text = " ".join([para.get_text() for para in paragraphs])
    return text

url = "https://pt.wikipedia.org/wiki/Microsoft_Office"

documento = extraindo_texto_da_web(url)


# Usando o Transformers

## Q & R
#### Caso de Uso:
  1. Precisamos de um modelo que entenda regras e história da Microsoft Office 365 responda nossas dúvidas sobre o assunto.

In [ ]:


qa = pipeline("question-answering")

pergunta = input("Pergunta: ")
result = qa(question=pergunta,
            context=documento,
            max_lenght= 100)
print(f"Resposta: {result['answer']}\n")


## Summarize

#### Caso de Uso:
  1. Precisamos de um modelo que entenda regras e história da Microsoft Office 365 e resuma o assunto

In [ ]:
resumo = pipeline("summarization")

result = resumo(documento,
                max_length=100,
                min_length=50,
                do_sample=False)

# Exibe o resumo
print(f"Resumo: {result[0]['summary_text']}\n")

## Análise de Sentimento

In [ ]:
analise_sentimento = pipeline("sentiment-analysis")

# Exemplo de texto para análise
paragrafo = """
Microsoft Office é uma suíte de aplicativos para escritório que contém programas como Word, Excel e PowerPoint.
Lançado pela Microsoft, é amplamente utilizado para criação e edição de documentos, planilhas e apresentações.
Desde sua introdução, tem sido uma ferramenta essencial tanto para o uso pessoal quanto para empresas.
No entanto, existem alternativas gratuitas, como o Google Docs, que têm desafiado o domínio da Microsoft neste setor.
"""

# Realiza a análise de sentimento no texto
result = analise_sentimento(paragrafo)

# Exibe os resultados
for idx, sentimento in enumerate(result):
    print(f"Frase {idx + 1}: {sentimento}\n")


# Usando o BERT

## Q & R

In [ ]:

qa = pipeline("question-answering", model="bert-large-uncased-whole-word-masking-finetuned-squad")

pergunta = input("Pergunta: ")
result = qa(question=pergunta,
            context=documento,
            max_lengh=100)

print(f"Resposta: {result['answer']}\n")

## Summarize

In [ ]:


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = BertTokenizerFast.from_pretrained('mrm8488/bert-mini2bert-mini-finetuned-cnn_daily_mail-summarization')
model = EncoderDecoderModel.from_pretrained('mrm8488/bert-mini2bert-mini-finetuned-cnn_daily_mail-summarization').to(device)

def resumo(documento):
    inputs = tokenizer([documento], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)

    output = model.generate(input_ids, attention_mask=attention_mask)

    return f" Resumo: {tokenizer.decode(output[0], skip_special_tokens=True)}"

resumo(documento)


## Análise de Sentimento

In [ ]:

analise_sentimento = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")

documento = """
Microsoft Office é uma suíte de aplicativos para escritório que contém programas como Word, Excel e PowerPoint.
Lançado pela Microsoft, é amplamente utilizado para criação e edição de documentos, planilhas e apresentações.
Desde sua introdução, tem sido uma ferramenta essencial tanto para o uso pessoal quanto para empresas.
No entanto, existem alternativas gratuitas, como o Google Docs, que têm desafiado o domínio da Microsoft neste setor.
"""

resultados = analise_sentimento(documento)

def mapear_sentimento(label):
    estrelas = int(label.split()[0])
    if estrelas in [1, 2]:
        return "Negativo"
    elif estrelas == 3:
        return "Neutro"
    else:
        return "Positivo"

# Processa o resultado
for idx, resultado in enumerate(resultados):
    sentimento = mapear_sentimento(resultado['label'])
    print(f"Texto {idx + 1}: {documento}")
    print(f"Classificação: {sentimento} (Confiança: {resultado['score']:.2f})\n")



##### inferência

Modelo Usado:

    O modelo nlptown/bert-base-multilingual-uncased-sentiment é uma variante do BERT treinada para análise de sentimentos em vários idiomas. Ele retorna a probabilidade de diferentes níveis de sentimento (de 1 a 5 estrelas).
        1-2 estrelas: "Negativo".
        3 estrelas: "Neutro".
        4-5 estrelas: "Positivo".

# Usando o OPENAI

In [ ]:
openai.api_key = "Use a sua API"


## Q & R

In [ ]:


pergunta = input("Faça a sua pergunta: ")

prompt = f"Contexto: {documento}\nPergunta: {pergunta}\nResposta:"

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "Você é um assistente útil."},
        {"role": "user", "content": prompt},
    ],
    max_tokens=50,
    temperature=0.0
)

# Exibe a resposta da pergunta
resposta = response['choices'][0]['message']['content'].strip()
print(f"Resposta: {resposta}")

## Summarize

In [ ]:
prompt = f"Resuma o seguinte texto de forma concisa:\n{documento}"

# Solicitação para o modelo GPT-3.5-turbo para resumir o texto
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "Você é um assistente que pode resumir textos."},
        {"role": "user", "content": prompt},
    ],
    max_tokens=100,
    temperature=0.7
)

# Exibe o resumo
resumo = response['choices'][0]['message']['content'].strip()
print(f"Resumo: {resumo}")

## Análise de Sentimento

In [ ]:


# Texto para análise de sentimento
texto = """
Maycon é um cara muito legal, eu amo ele!
"""

# Prompt para análise de sentimento
prompt = f"Classifique o sentimento do seguinte texto como 'Positivo', 'Negativo' ou 'Neutro':\n{texto}"

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",  # Ou "gpt-3.5-turbo" ou outro modelo de chat compatível
    messages=[
        {"role": "system", "content": "Você é um assistente útil."},
        {"role": "user", "content": prompt},
    ],
    max_tokens=60,
    temperature=0.0  # Temperatura baixa para respostas diretas
)

# Extrair e exibir o sentimento
sentimento = response['choices'][0]['message']['content'].strip()
print(f"Sentimento: {sentimento}")
